# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat publications.tsv










1/13/20	Universal Properties of Concentration Sensing in Ligand-Receptor Networks	Physical Review Letter	This paper shows universal properties of ligand receptor networks.	"Singh, V., & Nemenman, I. (2020). Universal properties of concentration sensing in large ligand-receptor networks. Physical Review Letters, 124(2), 028101."	paper-title-number-9	https://doi.org/10.1103/PhysRevLett.124.028101

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [9]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,4/17/12,Ordinary Percolation with Discontinuous Transi...,Nature Communications,This paper gives an analytical proof of the ex...,"Boettcher, S., Singh, V., & Ziff, R. M. (2012)...",paper-title-number-1,https://doi: 10.1038/ncomms1774 (2012)
1,7/18/14,Scaling of Clusters near Discontinuous Percola...,Physical Review E,This paper discusses percolation on hierarchic...,"Singh, V., & Boettcher, S. (2014). Scaling of ...",paper-title-number-2,https://doi.org/10.1103/PhysRevE.90.012117
2,10/17/14,Director Field Model of the Primary Visual Cor...,PLOS ONE,This paper develops a continumm model of the p...,"Singh, V., Tchernookov, M., Butterfield, R., &...",paper-title-number-3,https://doi.org/10.1371/journal.pone.0108991
3,11/13/14,From Explosive to Infinite-Order Transitions o...,Physical Review E,This paper studies Potts model on hierarchical...,"Singh, V., Brunson, C. T., & Boettcher, S. (20...",paper-title-number-4,https://doi.org/10.1103/PhysRevE.90.052119
4,8/31/16,Effects of Receptor Correlations on Molecular ...,Physical Review E,This paper shows that negative correlations do...,"Singh, V., Tchernookov, M., & Nemenman, I. (20...",paper-title-number-5,https://doi.org/10.1103/PhysRevE.94.022425
5,4/14/17,Simple Biochemical Networks allow Accurate Sen...,PLOS Comp Bio,This paper shows how one receptor can detect m...,"Singh, V., & Nemenman, I. (2017). Simple bioch...",paper-title-number-6,https://doi.org/10.1371/journal.pcbi.1005490
6,12/1/18,Computational Luminance Constancy from Natural...,Journal of Vision,This paper studies lightness constancy using a...,"Singh, V., Cottaris, N. P., Heasly, B. S., Bra...",paper-title-number-7,https://doi.org/10.1167/18.13.19
7,5/7/19,Competitive binding predicts nonlinear respons...,Proceedings of the National Academy of Sciences,This paper shows that olfacotry receptor repon...,"Singh, V., Murphy, N. R., Balasubramanian, V.,...",paper-title-number-8,https://doi.org/10.1073/pnas.1813230116
8,1/13/20,Universal Properties of Concentration Sensing ...,Physical Review Letter,This paper shows universal properties of ligan...,"Singh, V., & Nemenman, I. (2020). Universal pr...",paper-title-number-9,https://doi.org/10.1103/PhysRevLett.124.028101


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [10]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [13]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [14]:
!ls ../_publications/

12-paper-title-number-1.md 17-paper-title-number-6.md
14-paper-title-number-2.md 18-paper-title-number-7.md
14-paper-title-number-3.md 19-paper-title-number-8.md
14-paper-title-number-4.md 20-paper-title-number-9.md
16-paper-title-number-5.md


In [16]:
!cat ../_publications/12-paper-title-number-1.md

---
title: "Ordinary Percolation with Discontinuous Transitions "
collection: publications
permalink: /publication/4/17/12-paper-title-number-1
excerpt: 'This paper gives an analytical proof of the existence of discontinuous percolation transition. '
date: 4/17/12
venue: 'Nature Communications'
paperurl: 'https://doi: 10.1038/ncomms1774 (2012)'
citation: 'Boettcher, S., Singh, V., &amp; Ziff, R. M. (2012). Ordinary percolation with discontinuous transitions. Nature Communications, 3(1), 1-5.'
---
This paper gives an analytical proof of the existence of discontinuous percolation transition. 

[Download paper here](https://doi: 10.1038/ncomms1774 (2012))

Recommended citation: Boettcher, S., Singh, V., & Ziff, R. M. (2012). Ordinary percolation with discontinuous transitions. Nature Communications, 3(1), 1-5.